In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

In [2]:
# --- Configurações ---

TEMPO_SIMULACAO = 150  # Número de "passos" de tempo
INTERVALO_ANIMACAO_MS = 100
NOME_ARQUIVO_ANIMACAO = "monitoramento_drift.gif"
LIMITE_ALERTA_DRIFT = 0.75
TEMPOS_DE_DRIFT = [50, 100]  # Passos de tempo onde o drift ocorrerá

In [3]:
# --- Simulação de Desempenho do Modelo ---

def simular_desempenho_drift_impactante(tempo_simulacao, tempos_de_drift, limite_alerta):
    """Simula o desempenho de um modelo com quedas dramáticas no data drift."""
    desempenho = np.zeros(tempo_simulacao)
    desempenho[0] = 0.92  # Desempenho inicial alto
    for t in range(1, tempo_simulacao):
        change = random.uniform(-0.005, 0.005)
        if t in tempos_de_drift:
            change -= random.uniform(0.08, 0.15)  # Queda BEM mais acentuada no drift
        desempenho[t] = np.clip(desempenho[t-1] + change, 0.5, 0.95)
    return pd.Series(desempenho)

df_desempenho = simular_desempenho_drift_impactante(TEMPO_SIMULACAO, TEMPOS_DE_DRIFT, LIMITE_ALERTA_DRIFT)

In [4]:
# 01 --- Geração da Animação ---

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, TEMPO_SIMULACAO)
ax.set_ylim(0.6, 1.0)
ax.set_xlabel("Tempo (Simulado)")
ax.set_ylabel("Acurácia")
ax.set_title("Monitoramento de Modelo com Data Drift")
linha, = ax.plot([], [], color='royalblue', linewidth=2, label='Acurácia')
linha_alerta, = ax.plot([0, TEMPO_SIMULACAO], [LIMITE_ALERTA_DRIFT, LIMITE_ALERTA_DRIFT], color='red', linestyle='--', label=f'Limite de Alerta ({LIMITE_ALERTA_DRIFT:.2f})')
ponto_alerta = ax.scatter([], [], color='red', s=100, alpha=0) # Inicialmente invisível
texto_alerta = ax.text(TEMPO_SIMULACAO * 0.05, 0.85, '', color='red', fontsize=12, fontweight='bold')
alerta_visivel = False # Variável para controlar a visibilidade do alerta

plt.close()

In [5]:
# 02 --- Geração da Animação ---

def init():
    linha.set_data([], [])
    ponto_alerta.set_offsets(np.array([[0, 0]]))
    ponto_alerta.set_alpha(0)
    texto_alerta.set_text('')
    return linha, ponto_alerta, texto_alerta

In [6]:
# 03 --- Geração da Animação ---

def animate(i):
    global alerta_visivel # Permite modificar a variável global

    x = df_desempenho.index[:i+1]
    y = df_desempenho.values[:i+1]
    linha.set_data(x, y)

    if not alerta_visivel and y[-1] < LIMITE_ALERTA_DRIFT:
        alerta_visivel = True # Ativa o alerta na primeira vez que a condição é atendida
        ponto_alerta.set_offsets(np.array([[x[-1], y[-1]]]))
        ponto_alerta.set_alpha(1)
        texto_alerta.set_text('⚠️ ALERTA DE DRIFT!')
    elif alerta_visivel: # Mantém o alerta visível depois de ativado
        ponto_alerta.set_offsets(np.array([[x[-1], y[-1]]]))
        ponto_alerta.set_alpha(1)
        texto_alerta.set_text('⚠️ ALERTA DE DRIFT!')
    else:
        ponto_alerta.set_alpha(0)
        texto_alerta.set_text('')

    return linha, ponto_alerta, texto_alerta

In [7]:
# Gerando e salvando a animação

ani = animation.FuncAnimation(fig, animate, frames=TEMPO_SIMULACAO, interval=INTERVALO_ANIMACAO_MS, blit=True)

ani.save(NOME_ARQUIVO_ANIMACAO, writer='ffmpeg')

print(f"Animação salva como {NOME_ARQUIVO_ANIMACAO}")

Animação salva como monitoramento_drift.gif
